#### The portfolio strategies for now include:

    - Minimum Variance
        you can specify the number of periods to consider for the rolling window of covariance matrices
    - Equally weighted
        you can specify when the rebalancing will happen (from x to x periods)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import yfinance as yf


def MinimizePortfolioVariance(CovarReturns, RiskFreeRate, PortfolioSize):
    
    # define maximization of Sharpe Ratio using principle of duality
    def  f(x, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) ) #Portfolio stdev is sqrt(((weights @ covmat) @ weights.T))
        func = funcDenomr
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1 #the sum of the weights will be equal to 1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize) #will start with weights being 0.33
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0 #no shortsale constraint
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)

    return opt



def MinimumVarianceWeights(PriceMatrix, window=12,Rf=0):

    HPR_matrix = (PriceMatrix / PriceMatrix.shift(1) - 1).dropna(how='all') - Rf # creating the Holding period excess returns matrix. IMPORTANT, Rf must be in the same measure as returns. for exemple, if returns are daily, and rf is 1% yearly, you should place 0.01/252, or just ignore
    
    Weights=pd.DataFrame(columns=PriceMatrix.columns,index=PriceMatrix.index[window:]) # creating an empty weights dataframe

    for k in tqdm(range(len(HPR_matrix)-window)): 
        
        StocksWOnan=HPR_matrix.iloc[k:k+window,:].dropna(axis=1).columns #The StocksWOnan will make the function robust to periods where Stocks do not have prices
        PortfolioSize=len(StocksWOnan) #the number of columns with prices is the number of stocks we can invest in our portfolio
        CovMat=np.array(HPR_matrix[StocksWOnan].iloc[k:k+window,:].astype(float).cov()) #get the covariance matrix of the excess returns
        
        xOptimal =[]
        
        result = MinimizePortfolioVariance(CovMat, Rf, PortfolioSize) #minimize the portfolio variance
        xOptimal.append(result.x)
        
        xOptimalArray = np.array(xOptimal)
        
    
        Weights.loc[HPR_matrix.index[k+window],StocksWOnan]= xOptimalArray
        
        del StocksWOnan, PortfolioSize, CovMat
    
    return Weights



def EquallyWeightedPortfolio(PriceMatrix,rebalancing_period=20):
    

    Weights=pd.DataFrame(columns=PriceMatrix.columns,index=PriceMatrix.index)
    
    StocksWOnan=PriceMatrix.iloc[0,:].dropna().index # stocks without nan at the start of the data
    portfolioSize=len(StocksWOnan) #number of stocks to equally weigh at the beginning

    for k in tqdm(range(len(PriceMatrix))):
        if k % rebalancing_period == 0: # If we are in a rebalancing period
            StocksWOnan=PriceMatrix.iloc[k,:].dropna().index #stocks without nan in the analysed rebalancing period
            portfolioSize=len(StocksWOnan)
            Weights.loc[PriceMatrix.index[k],StocksWOnan]= 1/portfolioSize
        else: #if we are not in a rebalancing period
            Weights.loc[PriceMatrix.index[k],StocksWOnan]= 1/portfolioSize
    
    return Weights

# Showing an example

In [ ]:
def getPrices(tickers,start_date,end_date):

    Prices = pd.DataFrame()

    for i in tickers:
        Prices = pd.concat([Prices,yf.download(i, start=start_date, end=end_date)['Close']],axis=1)
    Prices.columns = tickers

    return Prices

# Set the start and end dates for the price data
start_date = '2010-01-01'  # Replace with your desired start date
end_date = '2022-01-01'  # Replace with your desired end date
tickers = ['AMZN','MSFT','TSLA','GOOG','META','JPM','NVDA']

PricesMatrix = getPrices(tickers,start_date,end_date)

In [ ]:
PricesMatrix

In [ ]:
MinVarianceWeights = MinimumVarianceWeights(PricesMatrix, window=10,Rf=0) #The window parameter gives the nº of periods to use the covariance matrix
MinVarianceWeights

In [ ]:
EquallyWeightedWeights = EquallyWeightedPortfolio(PricesMatrix,rebalancing_period=20) #The portfolio will be rebalanced each rebalancing_periods.if daily data, and rebalancing_periods=20, then the weights will be re adjusted every 20 days
EquallyWeightedWeights 